In [5]:
import random
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from decision_tree_cls import MyTreeClf

In [7]:
class MyForestClf:
    def __init__(self, n_estimators=10, max_features=0.5,
                max_samples=0.5, random_state=42,
                max_depth=5, min_samples_split=2,
                max_leafs=20, bins=16, oob_score=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_samples = max_samples
        self.random_state = random_state
        
        # Параметры деревьев
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.bins = bins
        
        # Лес (список деревьев)
        self.trees = []
        
        # Переменная для хранения количества листьев
        self.leafs_cnt = 0
        
        # Словарь для хранения важности фичей
        self.fi = {}
        
        self.oob_score = oob_score  # Новое поле для выбора метрики
        self.oob_score_ = None
    
    def fit(self, X: pd.DataFrame, y: pd.Series):
        # Фиксируем сид для воспроизводимости
        random.seed(self.random_state)
        
        # Инициализация параметров
        init_cols = list(X.columns)  # Все колонки
        init_rows_cnt = len(X)  # Количество строк
        
         # Инициализируем важность фичей
        self.fi = {col: 0 for col in init_cols}
        
         # Для OOB оценок
        # Списки для хранения OOB-выборок
        oob_predictions = np.zeros((init_rows_cnt, self.n_estimators))  # Для хранения OOB предсказаний
        oob_counts = np.zeros(init_rows_cnt,  dtype=np.int32)  # Считаем сколько раз каждая строка попадает в OOB выборку
        
        # Проходим по количеству деревьев
        for i in range(self.n_estimators):
            # Выбираем случайные колонки
            cols_smpl_cnt = round(self.max_features * len(init_cols))  # Количество колонок
            cols_idx = random.sample(init_cols, cols_smpl_cnt)  # Индексы колонок
            
            # Выбираем случайные строки
            rows_smpl_cnt = round(self.max_samples * init_rows_cnt)  # Количество строк
            rows_idx = random.sample(range(init_rows_cnt), rows_smpl_cnt)  # Индексы строк
            
            # OOB выборка (все строки, которые не попали в обучающую выборку)
            oob_idx = [idx for idx in range(init_rows_cnt) if idx not in rows_idx]
            
            # Формируем подвыборку данных
            X_subset = X.iloc[rows_idx][cols_idx]
            y_subset = y.iloc[rows_idx]
            
            # Создаем дерево с импортированным классификатором дерева
            tree = MyTreeClf(max_depth=self.max_depth, 
                             min_samples_split=self.min_samples_split, 
                             max_leafs=self.max_leafs, 
                             bins=self.bins)
            
            # Обучаем дерево
            tree.fit(X_subset, y_subset)
            
            # Добавляем дерево в лес
            self.trees.append(tree)
            
            # Предсказание для OOB выборки
            if len(oob_idx) > 0:
                oob_pred = tree.predict_proba(X.iloc[oob_idx])
                oob_predictions[oob_idx, i] = oob_pred  # Сохраняем предсказания
                oob_counts[oob_idx] += 1  # Увеличиваем счетчик OOB для этих строк
            
            # Подсчитываем количество листьев
            self.leafs_cnt += tree.leaf_count()
            
             # Обновляем важность признаков
            for col in cols_idx:
                self.fi[col] += tree.feature_importances()[col]
        
        # Усредняем предсказания по OOB строкам
        valid_oob_mask = oob_counts > 0
        if np.any(valid_oob_mask):
            oob_avg_predictions = np.mean(oob_predictions[valid_oob_mask], axis=1)
            oob_avg_labels = (oob_avg_predictions > 0.5).astype(int)

            # Вычисление метрики
            if self.oob_score == 'accuracy':
                self.oob_score_ = accuracy_score(y[valid_oob_mask], oob_avg_labels)
            elif self.oob_score == 'precision':
                self.oob_score_ = precision_score(y[valid_oob_mask], oob_avg_labels)
            elif self.oob_score == 'recall':
                self.oob_score_ = recall_score(y[valid_oob_mask], oob_avg_labels)
            elif self.oob_score == 'f1':
                self.oob_score_ = f1_score(y[valid_oob_mask], oob_avg_labels)
            elif self.oob_score == 'roc_auc':
                self.oob_score_ = roc_auc_score(y[valid_oob_mask], oob_avg_predictions)
            
    def predict(self, X: pd.DataFrame, type: str = 'mean'):
        predictions = []
        
        for i in range(len(X)):
            # Получаем предсказания всех деревьев для одной строки
            tree_predictions = [tree.predict(X.iloc[[i]])[0] for tree in self.trees]
            
            if type == 'mean':
                # Усредняем вероятности и применяем порог 0.5 для бинарного предсказания
                mean_pred = np.mean(tree_predictions)
                final_pred = 1 if mean_pred > 0.5 else 0
            elif type == 'vote':
                # Подсчитываем голосование деревьев
                votes = np.bincount(tree_predictions)
                final_pred = np.argmax(votes) if len(votes) == 1 or votes[0] != votes[1] else 1
            else:
                raise ValueError("Parameter 'type' must be 'mean' or 'vote'")
            
            predictions.append(final_pred)
        
        return predictions

    def predict_proba(self, X: pd.DataFrame):
        probabilities = []
        
        for i in range(len(X)):
            # Получаем вероятности от каждого дерева для одной строки
            tree_probs = [tree.predict_proba(X.iloc[[i]]) for tree in self.trees]
            
            # Усредняем вероятности
            mean_prob = np.mean(tree_probs)
            probabilities.append(mean_prob)
        
        return probabilities
    
    def feature_importances(self):
        return self.fi


In [8]:
# Тестирование OOB error
X, y = make_classification(n_samples=100, n_features=5, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
y = pd.Series(y)

# Обучение модели
forest = MyForestClf(n_estimators=3, oob_score='accuracy')и
forest.fit(X, y)

# Вывод OOB score
print(f"OOB accuracy: {forest.oob_score_}")

OOB accuracy: 0.6781609195402298


In [4]:
# Тестирование важности фичей
X, y = make_classification(n_samples=100,  # Количество строк (сэмплов)
                           n_features=4,   # Количество признаков
                           n_informative=3,  # Количество информативных признаков
                           n_redundant=0,  # Количество избыточных признаков
                           n_classes=2,  # Количество классов (бинарная классификация)
                           random_state=42)  # Для воспроизводимости
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(4)])  # Исправлено на 4
y = pd.Series(y)

# Создание и обучение случайного леса
forest = MyForestClf(n_estimators=5, max_features=0.6, max_samples=0.7, random_state=42)
forest.fit(X, y)

# Получение важности всех признаков
print("All feature importances:", forest.feature_importances())

All feature importances: {'feature_0': 1.4338941410209771, 'feature_1': 1.3199058999517281, 'feature_2': 1.1477802213853479, 'feature_3': 0}


In [7]:
#Тестирование предсказания
X, y = make_classification(n_samples=100,  # Количество строк (сэмплов)
                           n_features=4,   # Количество признаков
                           n_informative=3,  # Количество информативных признаков
                           n_redundant=0,  # Количество избыточных признаков
                           n_classes=2,  # Количество классов (бинарная классификация)
                           random_state=42)  # Для воспроизводимости

# Преобразуем X в DataFrame для удобства
X = pd.DataFrame(X, columns=[f'feature{i}' for i in range(1, 5)])

# Инициализация модели случайного леса
forest_clf = MyForestClf(n_estimators=5, max_features=0.6, max_samples=0.7, random_state=42)

# Обучение модели на наборе данных
forest_clf.fit(X, pd.Series(y))

# Предсказания классов (mean и vote)
predictions_mean = forest_clf.predict(X, type='mean')
predictions_vote = forest_clf.predict(X, type='vote')

# Предсказания вероятностей
probabilities = forest_clf.predict_proba(X)

# Проверим результаты
print(f"Сумма предсказаний (type='mean'): {sum(predictions_mean)}")
print(f"Сумма голосов (type='vote'): {sum(predictions_vote)}")
print(f"Сумма вероятностей: {sum(probabilities)}")

Сумма предсказаний (type='mean'): 45
Сумма голосов (type='vote'): 45
Сумма вероятностей: 45.0


In [3]:
#Тестирование обучения
X, y = make_classification(n_samples=100,  # Количество строк (сэмплов)
                           n_features=4,   # Количество признаков
                           n_informative=3,  # Количество информативных признаков
                           n_redundant=0,  # Количество избыточных признаков
                           n_classes=2,  # Количество классов (бинарная классификация)
                           random_state=42)  # Для воспроизводимости

# Преобразуем X в DataFrame для удобства
X = pd.DataFrame(X, columns=[f'feature{i}' for i in range(1, 5)])

# Инициализация модели случайного леса
forest_clf = MyForestClf(n_estimators=5, max_features=0.6, max_samples=0.7, random_state=42)

# Обучение модели на наборе данных
forest_clf.fit(X, pd.Series(y))

# Вывод количества листьев после обучения
print(f"Количество листьев во всем лесу: {forest_clf.leafs_cnt}")

Количество листьев во всем лесу: 55


In [6]:
# Тестирование класса
forest1 = MyForestClf()
forest2 = MyForestClf(n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16)
forest3 = MyForestClf(n_estimators=8, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16)

# Проверка
print(forest1)
print(forest2)
print(forest3)

MyForestReg class: n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=2, max_leafs=20, bins=16, criterion=entropy
MyForestReg class: n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16, criterion=entropy
MyForestReg class: n_estimators=8, max_features=0.5, max_samples=0.5, random_state=42, max_depth=5, min_samples_split=5, max_leafs=20, bins=16, criterion=entropy
